This is the jupiter notebook that Zhou writes some test functions

In [1]:
from functions import *
from optimize_obj import *
from Optalg import *
from runcase import *

In [3]:
import pandas as pd

df = pd.read_csv('result/Quadratic_BFGS_Armijo.txt', delim_whitespace=True)
df

,Iter,fval,||grad||,alpha
0,0,43.692131,47.765528,0.000000
1,1,16.126107,29.347442,0.062500
2,2,4.658875,17.729111,1.604220
3,3,2.364382,14.428424,1.855536
4,4,-1.048706,6.880454,0.535038
5,5,-2.499760,3.105667,1.344696
6,6,-3.769741,4.849286,5.525277
7,7,-5.942284,4.939172,1.003717
8,8,-9.647835,3.240647,0.374821
9,9,-14.008957,0.856669,0.835142


This is for the problem 1

In [2]:
# dimension n = 10
n = 10
# randomly generate a symmetric positive definite matrix Q, and q
Q = np.random.rand(n, n)
Q = Q @ Q.T
q = np.random.rand(n)

# declare the object of the class optimize_obj
obj = quad_obj(Q, q)

# solve the objective function using the gradient descent method
x0 = np.random.rand(n)

runcase(obj, x0, Objname = 'Quadratic', line_search_method = 'Armijo', make_table = True)

Objective function info:
problem dim =  10
method =  gradient_descend
Line search options:
Line search method =  Armijo
alphainit =  1
beta =  0.5
c1 =  0.0001
minstep =  1e-13
Convergence options:
ftol =  1e-10
gtol =  1e-10
xtol =  1e-10
maxiter =  1000
Iter	fval		||grad||	alpha
0	43.69213078	47.76552832	0.00000000
[15.68817739 12.41575243 16.20466433 16.17353244 13.10186206 16.58085643
 16.14369997  9.37113623 17.73929913 15.72889105]
[-15.68817739 -12.41575243 -16.20466433 -16.17353244 -13.10186206
 -16.58085643 -16.14369997  -9.37113623 -17.73929913 -15.72889105]
[15.68817739 12.41575243 16.20466433 16.17353244 13.10186206 16.58085643
 16.14369997  9.37113623 17.73929913 15.72889105]
[-15.68817739 -12.41575243 -16.20466433 -16.17353244 -13.10186206
 -16.58085643 -16.14369997  -9.37113623 -17.73929913 -15.72889105]
[15.68817739 12.41575243 16.20466433 16.17353244 13.10186206 16.58085643
 16.14369997  9.37113623 17.73929913 15.72889105]
[-15.68817739 -12.41575243 -16.20466433 -16.17

c:\Users\specf\anaconda3\envs\qids-2023-comp\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\specf\anaconda3\envs\qids-2023-comp\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the data from the text file
# end if encounter a blank line


# Plot the function value (fval) over iterations
plt.plot(data['Iter'], data['fval'])
plt.xlabel('Iteration')
plt.ylabel('Function Value (fval)')
plt.title('Function Value vs. Iteration')
plt.grid(True)
plt.show()

# Plot the gradient magnitude (||grad||) over iterations
plt.plot(data['Iter'], data['||grad||'])
plt.xlabel('Iteration')
plt.ylabel('Gradient Magnitude (||grad||)')
plt.title('Gradient Magnitude vs. Iteration')
plt.grid(True)
plt.show()

# Plot alpha over iterations
plt.plot(data['Iter'], data['alpha'])
plt.xlabel('Iteration')
plt.ylabel('Alpha')
plt.title('Alpha vs. Iteration')
plt.grid(True)
plt.show()

ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements